In [27]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace
from importlib import reload
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import sys
sys.path.append("/home/kim2712/Desktop/research/conformal_tree/")
#sys.path.append("/home/kim2712/Desktop/research/rlabc/")
#from rlabc.treeproposal import CARTTree
#CARTTree

In [29]:
import torch
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [30]:
from conformal_tree._nets import network_toy as nt
reload(nt)

<module 'conformal_tree._nets.network_toy' from '/home/kim2712/Desktop/research/conformal_tree/conformal_tree/_nets/network_toy.py'>

In [31]:
import conformal_tree._data.baseline_data as baseline_data

In [33]:
(x_train, y_train), (x_calib,y_calib) = baseline_data.get_mnist_np(device = DEVICE)#, n0=30000)
x_test,y_test = baseline_data.get_mnist_np(device = DEVICE, train=False)

In [34]:
from torch.utils.data import TensorDataset, DataLoader

In [35]:
dataset_train = TensorDataset(x_train, y_train)
dataset_calib = TensorDataset(x_calib, y_calib)
dataset_test = TensorDataset(x_test,y_test)

test_loader = DataLoader(dataset_test, batch_size=100, shuffle=True)
train_loader = DataLoader(dataset_train, batch_size=100, shuffle=True)
calib_loader = DataLoader(dataset_train, batch_size=100, shuffle=True)

In [45]:
net = nt.MLP(input_dim=28**2, out_dim=10, device = DEVICE, factor=4)#16) #128

In [46]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=0.02)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.95)

In [47]:
for epoch in range(10):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
            
    scheduler.step()
    #if it % 1000 ==0: 
    print(loss)

tensor(1.1410, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8979, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9800, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7181, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8713, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8458, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7380, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9097, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8701, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6628, device='cuda:0', grad_fn=<NllLossBackward>)


In [48]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 78 %


In [49]:
soft= torch.nn.Softmax(dim=1)
np.round(soft(outputs).detach().cpu(), decimals=2)

tensor([[0.0300, 0.0200, 0.1900, 0.0800, 0.0200, 0.2300, 0.1000, 0.0100, 0.2800,
         0.0300],
        [0.0000, 0.8600, 0.1000, 0.0200, 0.0000, 0.0000, 0.0100, 0.0000, 0.0100,
         0.0000],
        [0.0000, 0.7400, 0.1800, 0.0400, 0.0000, 0.0100, 0.0100, 0.0000, 0.0200,
         0.0000],
        [0.0000, 0.7400, 0.1700, 0.0400, 0.0000, 0.0100, 0.0200, 0.0000, 0.0200,
         0.0000],
        [0.0000, 0.6700, 0.1700, 0.0600, 0.0000, 0.0200, 0.0300, 0.0000, 0.0400,
         0.0000],
        [0.0000, 0.0000, 0.0200, 0.0000, 0.0300, 0.0100, 0.8900, 0.0000, 0.0500,
         0.0000],
        [0.0300, 0.0000, 0.0100, 0.8400, 0.0000, 0.0400, 0.0000, 0.0600, 0.0200,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.6800, 0.0200, 0.0200, 0.0000, 0.0100,
         0.2600],
        [0.6400, 0.0000, 0.0000, 0.0100, 0.0000, 0.1900, 0.0000, 0.0100, 0.0600,
         0.0900],
        [0.0000, 0.0100, 0.2900, 0.0000, 0.0200, 0.0300, 0.5500, 0.0000, 0.0900,
         0.0000],
        [0

In [50]:
import pathlib
path = "./trained_nets"
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
#torch.save(net.state_dict(),  path+F"/MNIST.net")
#torch.save(net.state_dict(),  path+F"/MNIST_half.net")
#torch.save(net.state_dict(),  path+F"/MNIST_smallnet_half.net")
torch.save(net.state_dict(),  path+F"/MNIST_minimum.net")